In [ ]:
import os

from diffusers import AutoencoderKLMochi
from mochi_pipeline import MochiPipeline, linear_quadratic_schedule, retrieve_timesteps
from transformers import T5EncoderModel, T5Tokenizer
from mochi_processor import MochiAttnProcessor2_0
import torch
import torchvision
from pathlib import Path
from diffusers.utils import export_to_video


def encode_videos(model: torch.nn.Module, vid_path: Path):
    video, _, metadata = torchvision.io.read_video(str(vid_path), output_format="THWC", pts_unit="secs")
    video = video.permute(3, 0, 1, 2)
    og_shape = video.shape
    
    print(f"Original video shape: {og_shape}")
    video = video.unsqueeze(0)
    video = video.float() / 127.5 - 1.0
    video = video.to(model.device)

    assert video.ndim == 5

    with torch.inference_mode():
        with torch.autocast("cuda", dtype=torch.bfloat16):
            ldist = model._encode(video)
    return ldist


In [ ]:
model_id = "genmo/mochi-1-preview"
pipeline = MochiPipeline.from_pretrained(model_id, torch_dtype=torch.float16)

pipeline.enable_vae_slicing()
pipeline.enable_vae_tiling()
pipeline.to("cuda")
# import numpy as np
# tokenizer = pipeline.tokenizer
# prompt = "a camflagued *animal* moving in the +background+"
# positive_start_index = tokenizer.tokenize(prompt).index("▁*")
# positive_end_index = tokenizer.tokenize(prompt).index("*")

# negative_start_index = tokenizer.tokenize(prompt).index("▁+")
# negative_end_index = tokenizer.tokenize(prompt).index("+")

# print(tokenizer.tokenize(prompt))


# indices = torch.tensor(list(range(positive_start_index + 1, positive_end_index)) + list(range(negative_start_index + 1, negative_end_index)))# + 1 # plus one because of the <extra_id_0> token
# print(indices)
# positive_mask = torch.tensor([1] * (positive_end_index - positive_start_index - 1) + [0] * (negative_end_index - negative_start_index - 1))
# negative_mask = torch.tensor([0] * (positive_end_index - positive_start_index - 1) + [1] * (negative_end_index - negative_start_index - 1))
# print(positive_mask)
# print(negative_mask)

# threshold_noise = 0.025
# sigmas = linear_quadratic_schedule(20, threshold_noise)
# sigmas = np.array(sigmas)
# timesteps = retrieve_timesteps(
#             pipeline.scheduler,
#             num_inference_steps=20,
#             device="cuda",
#             timesteps=None,
#             sigmas=sigmas,
#         )

# for block in pipeline.transformer.transformer_blocks:
#     block.attn1.processor = MochiAttnProcessor2_0(token_index_of_interest=indices, positive_mask=positive_mask) 

# video = pipeline(#latents=noise.cuda(), 
#         #  timesteps=timesteps[0].cpu(),#[:15].cpu(),
#          num_inference_steps=10,
#          prompt="a cat", 
#          negative_prompt="this is a nagative prompt",
#          emphasize_indices=(0,0), 
#          num_frames=12,
#          emphasize_neg_indices=(0,0),
) 

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 94.50 GiB of which 54.06 MiB is free. Including non-PyTorch memory, this process has 91.75 GiB memory in use. Of the allocated memory 90.23 GiB is allocated by PyTorch, and 832.54 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [30]:
with torch.inference_mode():
    video = "/home/ubuntu/wash/cog_dataset/crab_1_001.mp4"
    latents = encode_videos(
        pipeline.vae,
        Path(video),
    )[:,:12].to(pipeline.vae.device).to(pipeline.vae.dtype)

/home/ubuntu/anaconda3/lib/python3.12/site-packages/torchvision/io/_video_deprecation_warning.py:5: UserWarning: The video decoding and encoding capabilities of torchvision are deprecated from version 0.22 and will be removed in version 0.24. We recommend that you migrate to TorchCodec, where we'll consolidate the future decoding/encoding capabilities of PyTorch: https://github.com/pytorch/torchcodec
  warnings.warn(
/home/ubuntu/anaconda3/lib/python3.12/site-packages/torchvision/io/video.py:199: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")


Original video shape: torch.Size([3, 81, 480, 848])


In [ ]:
pipeline.enable_vae_slicing()
pipeline.enable_vae_tiling()
pipeline.enable_model_cpu_offload()
from diffusers.utils import export_to_video


with torch.no_grad():
    has_latents_mean = hasattr(pipeline.vae.config, "latents_mean") and pipeline.vae.config.latents_mean is not None
    has_latents_std = hasattr(pipeline.vae.config, "latents_std") and pipeline.vae.config.latents_std is not None
    if has_latents_mean and has_latents_std:
        latents_mean = (
            torch.tensor(pipeline.vae.config.latents_mean).view(1, 12, 1, 1, 1).to(latents.device, latents.dtype)
        )
        latents_std = (
            torch.tensor(pipeline.vae.config.latents_std).view(1, 12, 1, 1, 1).to(latents.device, latents.dtype)
        )
        latents_ = latents * latents_std / pipeline.vae.config.scaling_factor + latents_mean
    else:
        latents_ = latents / pipeline.vae.config.scaling_factor

    video = pipeline.vae.decode(latents_, return_dict=False)[0]
    video = pipeline.video_processor.postprocess_video(video, output_type="pil")

export_to_video(
    video[0],
    "output.mp4",
    fps=30
)

'output.mp4'

In [ ]:
import numpy as np
tokenizer = pipeline.tokenizer
prompt = "a camflagued *animal* moving in the +background+"
positive_start_index = tokenizer.tokenize(prompt).index("▁*")
positive_end_index = tokenizer.tokenize(prompt).index("*")

negative_start_index = tokenizer.tokenize(prompt).index("▁+")
negative_end_index = tokenizer.tokenize(prompt).index("+")

print(tokenizer.tokenize(prompt))


indices = torch.tensor(list(range(positive_start_index + 1, positive_end_index)) + list(range(negative_start_index + 1, negative_end_index)))# + 1 # plus one because of the <extra_id_0> token
print(indices)
positive_mask = torch.tensor([1] * (positive_end_index - positive_start_index - 1) + [0] * (negative_end_index - negative_start_index - 1))
negative_mask = torch.tensor([0] * (positive_end_index - positive_start_index - 1) + [1] * (negative_end_index - negative_start_index - 1))
print(positive_mask)
print(negative_mask)

threshold_noise = 0.025
sigmas = linear_quadratic_schedule(20, threshold_noise)
sigmas = np.array(sigmas)
timesteps = retrieve_timesteps(
            pipeline.scheduler,
            num_inference_steps=20,
            device="cuda",
            timesteps=None,
            sigmas=sigmas,
        )

for block in pipeline.transformer.transformer_blocks:
    block.attn1.processor = MochiAttnProcessor2_0(token_index_of_interest=indices, positive_mask=positive_mask) 

['▁', 'a', '▁cam', 'f', 'la', 'gue', 'd', '▁*', 'anim', 'al', '*', '▁moving', '▁in', '▁the', '▁+', 'back', 'ground', '+']
tensor([ 8,  9, 15, 16])
tensor([1, 1, 0, 0])
tensor([0, 0, 1, 1])


In [33]:
noise = torch.randn_like(latents)
# latents = (latents - latents_mean) / latents_std
# noisy_latents = pipeline.scheduler.scale_noise(latents, timesteps[0][15:16], noise) 

In [34]:
# pipeline.scheduler.num_inference_steps

In [35]:
# with torch.no_grad():
#     has_latents_mean = hasattr(pipeline.vae.config, "latents_mean") and pipeline.vae.config.latents_mean is not None
#     has_latents_std = hasattr(pipeline.vae.config, "latents_std") and pipeline.vae.config.latents_std is not None
#     if has_latents_mean and has_latents_std:
#         latents_mean = (
#             torch.tensor(pipeline.vae.config.latents_mean).view(1, 12, 1, 1, 1).to(latents.device, latents.dtype)
#         )
#         latents_std = (
#             torch.tensor(pipeline.vae.config.latents_std).view(1, 12, 1, 1, 1).to(latents.device, latents.dtype)
#         )
#         latents_ = noisy_latents * latents_std / pipeline.vae.config.scaling_factor + latents_mean
#     else:
#         latents_ = noisy_latents / pipeline.vae.config.scaling_factor

#     video = pipeline.vae.decode(latents_, return_dict=False)[0]
#     video = pipeline.video_processor.postprocess_video(video, output_type="pil")


In [36]:
export_to_video(
    video[0],
    "output.mp4",
    fps=30
)

'output.mp4'

In [ ]:
from diffusers.utils import export_to_video
video = pipeline(#latents=noise.cuda(), 
        #  timesteps=timesteps[0].cpu(),#[:15].cpu(),
         num_inference_steps=10,
         prompt=prompt, 
         negative_prompt="this is a nagative prompt",
         emphasize_indices=(0,0), 
         num_frames=12,
         emphasize_neg_indices=(0,0),
) 

(0, 0)


  0%|          | 0/10 [00:00<?, ?it/s]

/home/ubuntu/anaconda3/lib/python3.12/site-packages/diffusers/image_processor.py:147: RuntimeWarning: invalid value encountered in cast
  images = (images * 255).round().astype("uint8")


In [38]:
export_to_video(video["frames"][0], "output.mp4")

'output.mp4'

In [ ]:
frames = torchvision.io.read_video(str("/home/wg25r/make_it_move/training_free/finetrainers/training/mochi-1/dataset/crab_000.mp4"), output_format="THWC", pts_unit="secs")[0]
extracted_positive_maps = [] 
extracted_negative_maps = []
maps = pipeline.attention_maps  
for step in range(max(len(maps) - 10, 0), len(maps)-1):
    for layer in range(len(maps[step])): 
        # print(maps[step][layer].shape)# [B, H, K, Q]
        map = maps[step][layer][0].mean(0)[0] # use sum, because in it it was softmax and spread outed
        # print(maps[step][layer][0].mean(0)[positive_mask==1].shape) # need to use bool!
        extracted_positive_maps.append(map.cpu().float().numpy().reshape(-1, frames[0].shape[0]//16, frames[0].shape[1]//16))
        
        map = maps[step][layer][0].mean(0)[1]
        extracted_negative_maps.append(map.cpu().float().numpy().reshape(-1, frames[0].shape[0]//16, frames[0].shape[1]//16))

        # assert torch.all(maps[step][layer][0].mean(0).sum(0) >= 0.99), maps[step][layer][0].mean(0).sum(0)
extracted_positive_maps = np.array(extracted_positive_maps)
extracted_negative_maps = np.array(extracted_negative_maps)
np.isnan(extracted_positive_maps).any()

In [ ]:
import numpy as np
import math
file_id = 1
# %%
mask = (extracted_positive_maps > extracted_positive_maps.mean(axis=0) + extracted_positive_maps.std(axis=0)).astype(np.float32) + (extracted_positive_maps < extracted_positive_maps.mean(axis=0) - extracted_positive_maps.std(axis=0)).astype(np.float32)
mean_pos_map = np.nanmean(extracted_positive_maps, axis=0)


mask = (extracted_negative_maps > extracted_negative_maps.mean(axis=0) + extracted_negative_maps.std(axis=0)).astype(np.float32) + (extracted_negative_maps < extracted_negative_maps.mean(axis=0) - extracted_negative_maps.std(axis=0)).astype(np.float32)
mean_neg_map = np.nanmean(extracted_negative_maps, axis=0)

import cv2


def opening(x, kernel_size=3):
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))
    x = [cv2.morphologyEx(i, cv2.MORPH_OPEN, kernel) for i in x]
    return x

def blur(x, kernel_size=3):
    kernel = np.ones((kernel_size, kernel_size), np.float32) / (kernel_size * kernel_size)
    x = [cv2.filter2D(i, -1, kernel) for i in x]
    return x

def resize(x, size):
    return [cv2.resize(i, size) for i in x]
    
    
def normalize(x):
    return [(i - i.min()) / (i.max() - i.min()) for i in x]

def compose_frames(frames, maps):
    frames = [np.array(frame).astype(float)/255 for frame in frames]
    for i in range(len(frames)):
        map = cv2.cvtColor(maps[(i-2)//6], cv2.COLOR_GRAY2RGB).astype(float)       
        frames[i] = cv2.addWeighted(frames[i], 0.5, map, 0.5, 0)
    return frames

def repeat_maps(maps, total_len):
    export_maps = []
    for i in range(total_len):
        export_maps.append(maps[math.ceil((i-2)/6)])
    return export_maps

# print("a")
mean_pos_map = opening(mean_pos_map, kernel_size=3)
mean_neg_map = opening(mean_neg_map, kernel_size=3)
mean_pos_map = blur(mean_pos_map, kernel_size=3)
mean_neg_map = blur(mean_neg_map, kernel_size=3)
mean_pos_map = resize(mean_pos_map, size=(frames[0].shape[1], frames[0].shape[0]))
mean_neg_map = resize(mean_neg_map, size=(frames[0].shape[1], frames[0].shape[0]))

mean_pos_map = normalize(mean_pos_map)
mean_neg_map = normalize(mean_neg_map)

video = repeat_maps(mean_pos_map, len(frames))

# video = compose_frames(frames, mean_pos_map)
print(np.array(video).shape)
export_to_video(video, f"res/mochi_pos_{file_id:02d}_map.mp4", fps=30)

video = repeat_maps(mean_neg_map, len(frames))
# video = compose_frames(frames, mean_neg_map)
export_to_video(video, f"res/mochi_neg_{file_id:02d}_map.mp4", fps=30)

# fg = np.clip(np.array(mean_pos_map) - np.array(mean_neg_map), 0, 100)
fg = np.array(mean_pos_map) - np.array(mean_neg_map) * 2
fg[fg < 0] = 0
print(fg.max())

fg = normalize(fg)
video = compose_frames(frames, fg) 
export_to_video(video, f"map.mp4", fps=30)

In [ ]:
frames[0].shape, fg[0].shape